# <center>Assigments 11</center>  https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/

## Using a pre-trained Imagenet network to predict images into one of the 1000 Imagenet classes 

Fast.ai‘s Imagenette Dataset
Imagenette is a dataset that’s extracted from the large ImageNet collection of images. The reason behind releasing Imagenette is that researchers and students can practice on ImageNet-level images without needing that many computing resources.g:

### Step 1: Download the Imagenette dataset
### Step 2: Load images using ImageDataGenerator
### Step 3: Build a basic CNN model for image classification
### Step 4: Use transfer learning (VGG16) to improve accuracy

## Step 1: Download the Imagenette dataset
## https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

## Step 1: Download the Imagenette dataset
## https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

## Step 2: Load images using ImageDataGenerator


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications import VGG16
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [3]:
# create a new generator
imagegen = ImageDataGenerator()
# load train data
train = imagegen.flow_from_directory("imagenette2/train/", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))
# load val data
val = imagegen.flow_from_directory("imagenette2/val/", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))

Found 9469 images belonging to 10 classes.
Found 3927 images belonging to 10 classes.


In [4]:
pretrained_model = VGG16(include_top=False, weights='imagenet') 
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [ ]:
vgg_features_train = pretrained_model.predict(train) 
vgg_features_val = pretrained_model.predict(val)

47/74 [==================>...........] - ETA: 12:41

In [ ]:
train_target = to_categorical(train.labels) 
val_target = to_categorical(val.labels)

In [ ]:
model2 = Sequential()
model2.add(Flatten(input_shape=(7,7,512)))
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(10, activation='softmax'))

# compile the model
model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model2.summary()

# train model using features generated from VGG16 model
model2.fit(vgg_features_train, train_target, epochs=50, batch_size=128, validation_data=(vgg_features_val, val_target))

In [ ]:
# Plot accuracy graph                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Create an ImageDataGenerator for loading and preprocessing images
datagen = ImageDataGenerator()

# Load a batch of images from the dataset
batch_size = 10  # Number of images you want to load for prediction
sample_generator = datagen.flow_from_directory("imagenette2/val/", class_mode=None, shuffle=True, batch_size=batch_size, target_size=(224, 224))

# Get a batch of images and preprocess them
sample_images = next(sample_generator)
sample_images_features = pretrained_model.predict(sample_images)

# Predict using the model
predictions = model2.predict(sample_images_features)

# Get class labels for the predictions
predicted_classes = np.argmax(predictions, axis=1)

# Map class indices to class labels
class_labels = { 
    0: "tench",
    1: "springer",
    2: "casette_player",
    3: "chain_saw",
    4: "church",
    5: "French_horn",
    6: "garbage_truck",
    7: "gas_pump",
    8: "golf_ball",
    9: "parachute"
}
predicted_labels = [class_labels[index] for index in predicted_classes]

# Display the predicted class labels for the sample images
for i in range(batch_size):
    print("Predicted class label for sample image", i, ":", predicted_labels[i])
